# Compartimental Model Simulator

**Created by:** Samuel Ropert  
**Creation date:** 12/10/2020  
**Institution:** Computational Biology Lab - Fundación Ciencia y Vida, Chile  

## SEIR
This jupyter notebook shows how periodic examinations affect a pandemy evolution.


## Import Libraries

In [ ]:
import sys
from pathlib import Path
sys.path.insert(1, '../src/SEIR/')
sys.path.insert(1, '../src/utils/')

from datetime import datetime
import numpy as np
from numpy import linalg as LA
import pandas as pd
from time import time
import multiprocessing
from joblib import Parallel, delayed

import platform
OS = platform.system()

import matplotlib.pyplot as plt
if OS == 'Linux':    
    %matplotlib tk
    print('Linux')
elif OS == 'Windows': 
    %matplotlib qt
    print('Windows')
else:
    print('OS not detected :-|')


from class_SEIR2 import SEIR
from Quarantine import Quarantine
from Quarantine import Exams

## Epidemiological Parameters
* **beta:** Infection rate
* **mu:** Initial exposed obtained from the initial infected mu=E0/I0
* **Scale Factor:** Proportion of real infected compared to reported ones (1: all the infecteds are reported)
* **Sero Prevalence Factor:** Adjust the proportion of the population that enters the virus dynamics
* **Exposed Infection:** rate compared to the infected (0 the don't infect, 1 the infect in the same rate as the infected )

## Simulation Parameters

In [ ]:
beta = 0.2 # Contagion rate
mu = 1.5 # E0/I0 initial rate
SeroPrevalence = 1
expinfection = 0

# Simulation time
tsim = 500
# Population
population = 1000000
# Initial Active Infected 
I0 = 100

I_ac0 = 100
# Kinetic Saturation: 0 for mass action mixing
k_I=0
# Immunity Shield
k_R=0

### Quarantines

Quarantine object constructor:
 
 
 ```Quarantine(rem_mov,max_mov=0.85,qp=0,iqt=0,fqt=1000,movfunct = 'once')```
 
 * rem_mov: Remanent mobility during Quarantine
 * max_mov: Mobility during non quarantine periods
 * qp: Quarantine period (for dynamic quarantines)
 * iqt: Initial quarantine time
 * fqt: Final quarantine time
 * movfunct: Mobility function 

Mobility function types:
 * once: Total quarantine between iqt and fqt
 * square: Periodic quaratine with qp period

In [ ]:
s1 = Quarantine(1)

# Exams vs no Exams dynamics

In [ ]:
examrate = 20000 # Persons per day
period = 1
duty = 0.5 
psi = Exams(examrate,period,duty) # 

In [ ]:
# test
time = np.arange(0,30,0.1)
exams = [psi(t) for t in time]
plt.plot(time,exams)

## Create simulation Object

In [ ]:
simulation = SEIR(tsim=tsim,alpha=s1.alpha,psi = psi,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1)
# No exams dynamic for reference:
simulation_reference =  SEIR(tsim=tsim,alpha=s1.alpha,psi = 0,beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1)

## Simulate


In [ ]:
simulation.integr_sci(0,tsim,0.01)
simulation_reference.integr_sci(0,tsim,0.01)
print('simulation finished')

# Simulation Analysis

### SEIR Plot with Active infected

In [8]:
plt.plot(simulation.t,simulation.S,label='S',color = 'blue')
plt.plot(simulation.t,simulation.E,label='E',color = 'cyan')
plt.plot(simulation.t,simulation.I,label='I',color = 'red')
plt.plot(simulation.t,simulation.R,label='R',color = 'green')
plt.plot(simulation_reference.t,simulation_reference.S,label='S - No exams',color = 'blue',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.E,label='E - No exams',color = 'cyan',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.I,label='I - No exams',color = 'red',linestyle='dashed')
plt.plot(simulation_reference.t,simulation_reference.R,label='R - No exams',color = 'green',linestyle='dashed')

plt.title('Epidemiological Plot')
plt.legend(loc=0)
plt.show()

### Plot Accumulated Infected

In [9]:
plt.plot(simulation.t,simulation.I_ac,label='I_ac with exams')
plt.plot(simulation_reference.t,simulation_reference.I_ac,label='I_ac No Exams', linestyle='dashed')
plt.title('Accumulated Infected')
plt.legend(loc=0)
plt.show()

### Plot New Daily Infected

In [10]:
plt.plot(simulation.t,simulation.I_d,label='I_d with exams')
plt.plot(simulation_reference.t,simulation_reference.I_d,label='I_d no exams', linestyle='dashed')
plt.title('New Daily Infected')
plt.legend(loc=0)
plt.show()

### Amount of exams

In [11]:
plt.plot(simulation.t,simulation.e,label='Exams')
plt.plot(simulation_reference.t,simulation_reference.I_d,label='Exams',color = 'red')
plt.title('Amount of exams performed')
plt.legend(loc=0)
plt.show()

In [12]:
cost = 5
monthlycost = examrate*cost*30*duty
print('Monthly Cost: '+str(monthlycost)+' USD')

Monthly Cost: 1500000.0 USD


# Examination Strategies

In [13]:
examrate = [1000,5000,10000,20000,50000] # Persons per day
period = [1,2,5,10,15,30]
duty = 0.5
psi = [[Exams(i,j,duty) for j in period] for i in examrate] 

### Plot Exam campaings

In [28]:
time = np.arange(0,100,0.1)
exams = [[[psi[i][j](t) for t in time] for j in range(len(period))] for i in range(len(examrate))]

In [29]:
n = 3
time = np.arange(0,100,0.1)
exams = [[[psi[i][j](t) for t in time] for j in range(len(period))] for i in range(len(examrate))]
#plt.plot(time,exams[i][j])

fig, axs = plt.subplots(len(examrate), len(period))
for i in range(len(examrate)):
    for j in range(len(period)):
        axs[i,j].plot(time,exams[i][j],label='Examination Rate:'+str(examrate[i])+' Period: '+str(period[j]))
        axs[i,j].legend(loc=0)
    #axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))

## Simulate

In [14]:
# For parallel simulation
def simulate(simulation,j,tsim):
    simulation[j].integr_sci(0,tsim,0.1)
    return simulation[j]

In [18]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(period)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i][j],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1))
    sims.append(aux)

### Run simulation
Using parallel threads

In [19]:
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.6s
[Parallel(n_jobs=12)]: Done   2 out of   6 | elapsed:    1.7s remaining:    3.4s
[Parallel(n_jobs=12)]: Done   3 out of   6 | elapsed:    1.9s remaining:    1.9s
[Parallel(n_jobs=12)]: Done   4 out of   6 | elapsed:    2.2s remaining:    1.1s
[Parallel(n_jobs=12)]: Done   6 out of   6 | elapsed:    2.6s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   6 out of   6 | elapsed:    2.6s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    1.3s
[Parallel(n_jobs=12)]: Done   2 out of   6 | elapsed:    1.5s remaining:    3.0s
[Parallel(n_jobs=12)]: Done   3 out of   6 | elapsed:    1.6s remaining:    1.6s
[Parallel(n_jobs=12)]: Done   4 out of   6 | elapsed:    1.7s remaining:    0.8s
[Parallel(n_jobs=12)]: Done   6 out of   6 | elapsed:    2.2s remaining:    0.0s
[Pa

# Analysis
## Period analysis over a single amount of exam rate

In [20]:
j = -1

In [21]:
plt.plot(simulation_reference.t,simulation_reference.I_d,label='No exams',color = 'black',linestyle='dashed')
for i in range(len(period)):
    plt.plot(simulation[j][i].t,simulation[j][i].I_d,label='Examination Period: '+str(period[i]))
plt.legend(loc=0)
plt.title('New Daily Infected | '+str(examrate[j])+' exams/day')
plt.show()

## Amount of exams over a single period

In [22]:
j = 3 

In [24]:
plt.plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
for i in range(len(examrate)):
    plt.plot(simulation[i][3].t,simulation[i][3].I_d,label='Exams per day: '+str(examrate[i]))
plt.legend(loc=0)
plt.title('New Daily Infected | Campaing periods: '+str(period[j]))
plt.show()

## Gridplot

In [27]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(period)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(period)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination Period: '+str(period[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))


In [31]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(examrate)))
n = 3
fig, axs = plt.subplots(int(len(period)/n), n)
for j in range(len(period)):
    axs[int(j/n),j%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for i in range(len(examrate)):
        axs[int(j/n),j%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination Period: '+str(period[j]))
        axs[int(j/n),j%n].legend(loc=0)
    axs[int(j/n),j%n].set_title('Period: '+str(period[j]))


# Examrate vs duty

In [32]:
examrate = [1000,5000,10000,20000,50000] # Persons per day
period = 30
duty = [0.1,0.25,0.5,0.75,1]
psi = [[Exams(i,period,j) for j in duty] for i in examrate] 

In [37]:
# Define Simulations
sims = []
for i in range(len(examrate)):
    aux = []
    for j in range(len(duty)):
        aux.append(SEIR(tsim=tsim,alpha=s1.alpha,psi = psi[i][j],beta=beta,mu=mu,k_I=k_I,k_R=k_R,I0=I0,population=population,expinfection=0,SeroPrevFactor=1))
    sims.append(aux)

In [38]:
# Run simulation
num_cores = multiprocessing.cpu_count()
simulation = []
for i in range(len(examrate)):
    simulation.append(Parallel(n_jobs=num_cores, verbose=50)(delayed(simulate)(sims[i],j,tsim) for j in range(len(sims[i]))))

print('ready')

[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.1205s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.4s remaining:    0.5s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.4s remaining:    0.3s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.5s finished
[Parallel(n_jobs=12)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=12)]: Batch computation too fast (0.0758s.) Setting batch_size=2.
[Parallel(n_jobs=12)]: Done   2 out of   5 | elapsed:    0.3s remaining:    0.5s
[Parallel(n_jobs=12)]: Done   3 out of   5 | elapsed:    0.4s remaining:    0.2s
[Parallel(n_jobs=12)]: Done   5 out of   5 | elapsed:    0.4s remaining:    0.0s

In [39]:
colors = plt.cm.rainbow_r(np.linspace(0,1,len(duty)))
n = 3
fig, axs = plt.subplots(int(len(examrate)/n)+1, n)
for i in range(len(examrate)):
    axs[int(i/n),i%n].plot(simulation_reference.t,simulation_reference.I_d,label='I_d No exams',color = 'black',linestyle='dashed')
    for j in range(len(duty)):
        axs[int(i/n),i%n].plot(simulation[i][j].t,simulation[i][j].I_d,label='Examination duty: '+str(duty[j]))
        axs[int(i/n),i%n].legend(loc=0)
    axs[int(i/n),i%n].set_title('Exam Rate: '+str(examrate[i]))


# To Do 
## Sensitivity Analysis
## Add a new metric to study how the exams campaigns reduce the contagion rate.
Something like effective beta / Exams rate